# Analysis of period gradient in spreadouts
Paul Gerald Layague Sanchez (paul.sanchez@embl.de or pglsanchez@gmail.com) and Takehito Tomita (takehito.tomita@embl.de)

Last updated: 20200726 2110H

This script analyzes the period gradient in spreadouts. The input files for this script are:

*   period wavelet movie (dorsal side is up)
*   binary mask of the period wavelet movie

In [ ]:
# works for rotated samples, dorsal up, and mostly occupies middle of the field of view

import sys,os
from sys import argv, stdout
from os.path import expanduser
from pathlib import Path

import numpy as np
from os import path,walk

from skimage import io

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors

# import pandas as pd

import seaborn as sns

sns.set_style("ticks")
sns.despine()
sns.set_context("talk", font_scale = 1.5)

##v2: added rotation array. Place the degree of rotation for each sample.

chosen_colormap = plt.get_cmap('Blues') # for period gradient at different positions along 
chosen_colormap2 = 'viridis' # for evolution of period gradient for multiple samples

diameter = 50
dt = 10 # sampling time
frame_interval = 5 # interval between frames to be considered
min_period = 100 # minimum period for calculating true mean
start_position = 231 # start pixel position along vertical/DV axis
stop_position = 281 # stop pixel position along vertical/DV axis + 1
position_sampling = 1 

def figsave_to_tif(input_string):
    wdir = os.getcwd()
    print("Working in",wdir)

    p = Path(wdir)
    figure_names = list( p.glob(input_string) )
    if len(figure_names) == 0:
        print('Found no input movie.. exiting!')
        sys.exit(1)
    figure_names.sort()

    figure_path = os.path.join( wdir, figure_names[0].name )
    figure = io.imread(figure_path, plugin="tifffile")

    mat = np.zeros([len(figure_names),figure.shape[0],figure.shape[1],figure.shape[2]], dtype = np.uint8)
    print(mat.shape)

    for index,k in enumerate(figure_names):

        figure_name = k.name # the roi movie file name

        figure_path = os.path.join( wdir, figure_name )
        print('Opening :', figure_name)
        mat[index,:,:,:] = io.imread(figure_path, plugin="tifffile")
        os.remove(figure_path)

    out_name = input_string.replace('*','')[1:]
    out_path1 = os.path.join(wdir, 'flattened_'+out_name)
    io.imsave(out_path1, mat)
    print('written',out_path1)

def fig2save_to_tif(input_string):
    wdir = os.getcwd()
    print("Working in",wdir)

    p = Path(wdir)
    figure_names = list( p.glob(input_string) )
    if len(figure_names) == 0:
        print('Found no input movie.. exiting!')
        sys.exit(1)
    figure_names.sort()

    figure_path = os.path.join( wdir, figure_names[0].name )
    figure = io.imread(figure_path, plugin="tifffile")

    mat = np.zeros([len(figure_names),figure.shape[0],figure.shape[1],figure.shape[2]], dtype = np.uint8)
    print(mat.shape)

    for index,k in enumerate(figure_names):

        figure_name = k.name # the roi movie file name

        figure_path = os.path.join( wdir, figure_name )
        print('Opening :', figure_name)
        mat[index,:,:,:] = io.imread(figure_path, plugin="tifffile")
        os.remove(figure_path)

    out_name = input_string.replace('*','')[1:]
    out_path1 = os.path.join(wdir, 'mean_flattened_periodGradient'+out_name)
    io.imsave(out_path1, mat)
    print('written',out_path1)

def chop(array):
    ind = array > min_period # actual period values, not because of masked signal
    ind = np.where(ind)[0]
    chopped = np.array([np.array(ind),array[ind]])
    return chopped

def expand_zeros(array):
    ydim,xdim = array.shape
    for y in range(ydim):
        if np.prod(array[y,:])<1:
            array[y,:]=0
    return array

def extract_gradient(array,X,Y):
    #mask the array first
    w = 10    # width of sampling
    down_vector = np.mean(expand_zeros(array[int(Y):,int(X)-w:int(X)+w]),axis = 0)
    up_vector = np.flip(np.mean(expand_zeros(array[:int(Y),int(X)-w:int(X)+w]),axis = 0),axis = 0)
    left_vector = np.flip(np.mean(expand_zeros(array[int(Y)-w:int(Y)+w,:int(X)]),axis = 0),axis = 0)
    right_vector = np.mean(expand_zeros(array[int(Y)-w:int(Y)+w,int(X):]),axis = 0)
    vectors = [down_vector,up_vector,left_vector,right_vector]
    return vectors

wdir = os.getcwd()
print("Working in",wdir)

p = Path(wdir)

period_movie_names = list( p.glob('output_period*tif') ) # make list of period movies for gradient
period_movie_names.sort()
mask_movie_names = list( p.glob('mask*tif') )           #make list of intensity movies for fluorescent image
mask_movie_names.sort()

vector_allSamples = []

for index,k in enumerate(period_movie_names):


    period_movie_name = k.name # the roi movie file name
    mask_movie_name = mask_movie_names[index].name

    # open MAX movie as Mrm
    period_movie_path = os.path.join( wdir, period_movie_name )
    print('Opening :', period_movie_name)
    period_rm = io.imread(period_movie_path, plugin="tifffile")[:,:,:]    #tzcyx remove first channel

    mask_movie_path = os.path.join( wdir, mask_movie_name )
    mask_rm = io.imread(mask_movie_path, plugin="tifffile")[:,:,:]        #read intensity movie for fluorescent images, remove second channel(BF image)

    period_rm = period_rm*mask_rm


    NFrames, ydim, xdim = period_rm.shape
    
    vector_allFrames = []


    for frame in range(0,NFrames,frame_interval): # last number to specify intervals between frames to consider
        fig = plt.figure(figsize=(20,12))
        ax = fig.add_subplot(111)
        
        ax.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')
        vector_frame = []

        for row in range(start_position,stop_position,position_sampling): # change range to limit range in 512 pixel that will be considered
            print(row)
            vector = period_rm[frame,row,:]
            vector_frame.append([vector])
            ax.plot(vector,color = chosen_colormap(row), lw = 5, alpha = 0.3)
        
        vector_frame = np.array(vector_frame)
        vector_frame_mean = np.true_divide(vector_frame.sum(0),(vector_frame>100).sum(0)) # true mean, do not consider positions with no true values == masked/no signal
        vector_frame_mean = vector_frame_mean[0]
        vector_allFrames.append(vector_frame_mean)
        
        ax.plot(vector_frame_mean, color = 'magenta', lw = 10, alpha = 1.0)

        ax.plot([-1000,1000],[170,170],'k--',lw = 2,alpha = 0.5)
        ax.plot([-1000,1000],[150,150],'k--',lw = 2,alpha = 0.5)
        ax.plot([-1000,1000],[130,130],'k--',lw = 2,alpha = 0.5)
        ax.set_xlim([0,512])
        ax.set_ylim([100,250])
        ax.tick_params(axis='both', labelsize=30)
        ax.set_xlabel("SPACE [pixels]", labelpad = 10, fontsize = 30)
        ax.set_ylabel("PERIOD [mins]", labelpad = 10, fontsize = 30)
        fig.savefig(str(frame+1000)+period_movie_name[:-4]+'.tif')

    vector_allSamples.append(vector_allFrames)

    figsave_to_tif('1*'+period_movie_name[:-4]+'*.tif') 
    plt.close(fig)

    # plt.ylim([2*np.pi/200,2*np.pi/120])
    # plt.ylim([120,200])
    # fig1.savefig('frequency_gradient_'+str(index+1)+'.pdf')

In [ ]:
vector_allSamples = np.array(vector_allSamples) # sample, frame, position

In [ ]:
mean_samples = np.nanmean(np.where(vector_allSamples!=0,vector_allSamples,np.nan),0) # true mean, do not consider positions with no true values == masked/no signal

In [ ]:
for j in range(vector_allSamples.shape[1]):
  
  frame = j*frame_interval

  fig2 = plt.figure(figsize=(12,12))
  ax1 = fig2.add_subplot(111)
  ax1.set_title("time: "+str(frame*dt)+"min", fontsize = 30, loc = 'right')

  for i in range(vector_allSamples.shape[0]):

    ax1.plot(vector_allSamples[i,j,:], 'k--', lw = 3, alpha = 0.3)

  ax1.plot(mean_samples[j,:], 'm-', lw = 5, alpha = 1.0)
  ax1.plot([-1000,1000],[170,170],'k--',lw = 2,alpha = 0.5)
  ax1.plot([-1000,1000],[150,150],'k--',lw = 2,alpha = 0.5)
  ax1.plot([-1000,1000],[130,130],'k--',lw = 2,alpha = 0.5)
  ax1.set_xlim([0,512])
  ax1.set_ylim([100,250])
  ax1.tick_params(axis='both', labelsize=30)
  ax1.set_xlabel("SPACE [pixels]", labelpad = 10, fontsize = 30)
  ax1.set_ylabel("PERIOD [mins]", labelpad = 10, fontsize = 30)
  fig2.savefig(str(frame+2000)+period_movie_name[:-4]+'.tif', bbox_inches = 'tight')

fig2save_to_tif('2*.tif') 
plt.close(fig2)

In [ ]:
ncolors = mean_samples.shape[0]
sns.set_palette(chosen_colormap2, ncolors)

fig3 = plt.figure(figsize=(20,12))
ax2 = fig3.add_subplot(111)

for i in range(mean_samples.shape[0]):

  ax2.plot(mean_samples[i,:], lw = 5, alpha = 0.3)

ax2.plot([-1000,1000],[170,170],'k--',lw = 2,alpha = 0.5)
ax2.plot([-1000,1000],[150,150],'k--',lw = 2,alpha = 0.5)
ax2.plot([-1000,1000],[130,130],'k--',lw = 2,alpha = 0.5)
ax2.set_xlim([0,512])
ax2.set_ylim([100,250])
ax2.tick_params(axis='both', labelsize=30)
ax2.set_xlabel("SPACE [pixels]", labelpad = 10, fontsize = 30)
ax2.set_ylabel("PERIOD [mins]", labelpad = 10, fontsize = 30)

plt.savefig(str(wdir) + '/periodGradient_evolution.png', bbox_inches = 'tight')